In [1]:
%autosave 0

Autosave disabled


In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import text, create_engine
from pydataset import data

from env import get_connection

Let's read in the users and roles tables from the join_example_db.

In [3]:
engine = create_engine(get_connection('join_example_db'))

In [4]:
users_query = '''
              SELECT *
              FROM users
              '''

roles_query = '''
              SELECT *
              FROM roles
              '''

In [5]:
users = pd.read_sql(text(users_query), engine.connect())
roles = pd.read_sql(text(roles_query), engine.connect())

Let's try joining our two dataframes together with pd.concat()!

In [8]:
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


Ok, now how about calling the .merge() method off one of our tables!

In [9]:
pd.concat([users,roles]) # we concatonated row wise

,id,name,email,role_id
0,1,bob,bob@example.com,1.0
1,2,joe,joe@example.com,2.0
2,3,sally,sally@example.com,3.0
3,4,adam,adam@example.com,3.0
4,5,jane,jane@example.com,NaN
5,6,mike,mike@example.com,NaN
0,1,admin,NaN,NaN
1,2,author,NaN,NaN
2,3,reviewer,NaN,NaN
3,4,commenter,NaN,NaN


In [10]:
pd.concat ([users, roles], axis = 1) 

# we are concatonating column wise
#engineer some feature and return some array

,id,name,email,role_id,id,name
0,1,bob,bob@example.com,1.0,1.0,admin
1,2,joe,joe@example.com,2.0,2.0,author
2,3,sally,sally@example.com,3.0,3.0,reviewer
3,4,adam,adam@example.com,3.0,4.0,commenter
4,5,jane,jane@example.com,NaN,NaN,NaN
5,6,mike,mike@example.com,NaN,NaN,NaN


In [14]:
users.merge(roles, how='inner', left_on='role_id', right_on='id')

#i prefer to inner join the merge today . 
#left on is the name of the table on the left

,id_x,name_x,email,role_id,id_y,name_y
0,1,bob,bob@example.com,1.0,1,admin
1,2,joe,joe@example.com,2.0,2,author
2,3,sally,sally@example.com,3.0,3,reviewer
3,4,adam,adam@example.com,3.0,3,reviewer


#Let's read in the mpg dataset and create a new binary column for #automatic/manual transmission.

In [6]:
mpg = data('mpg')
mpg['auto_or_man'] = np.where(mpg['trans'].str.startswith('a'), 'auto', 'manual')
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,auto_or_man
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,auto
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,manual
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,manual
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,auto
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,auto


Let's do a crosstab of two categorical variables, the drive type and the transmission type.

In [15]:
pd.crosstab(mpg.drv, mpg.auto_or_man)

#crosstab expect two columns worth of information

auto_or_man,auto,manual
drv,,
4,75,28
f,65,41
r,17,8


We can set normalize equal to True, to return the proportion of all cars each combination represents.

A pivot table will look similar to a crosstab.

We specify which two categorical variables will be along the axes.

The index argument specifies the index and the columns argument specifies the column.

We will pass in a third argument, values, to return the mean of that column.

If we don't want the mean, we can specify a fourth argument (aggfunc) for a different aggregate function.

The map method will use a dictionary to map existing values to desired ones.

In [7]:
drv_dict = {'4': 'four-wheel drive',
            'f': 'front-wheel drive',
            'r': 'rear-wheel drive'}

We can transpose dataframes by saying .T.